# Minecraft treasure-hunt

In this notebook we'll attempt to find all treasure chests not facing the east. We received a zip-file with .nbt files, the file format of Minecraft. We'll use a python_nbt package to help us.

In [7]:
import python_nbt.nbt as nbt

 ## Exploration
I extracted one file to see what it looks like. I use the package to turn it into a JSON file for easier processing. 

In [23]:
file = nbt.read_from_nbt_file('mauron_eye_000.nbt')
jsonnbt = file.json_obj(full_json=False)

jsonnbt

{'size': [3, 4, 3],
 'entities': [],
 'blocks': [{'pos': [0, 0, 1], 'state': 0},
  {'pos': [1, 0, 0], 'state': 0},
  {'pos': [1, 0, 2], 'state': 0},
  {'pos': [2, 0, 1], 'state': 0},
  {'pos': [0, 2, 1], 'state': 0},
  {'pos': [1, 2, 0], 'state': 0},
  {'pos': [1, 2, 2], 'state': 0},
  {'pos': [2, 2, 1], 'state': 0},
  {'pos': [0, 0, 0], 'state': 1},
  {'pos': [0, 0, 2], 'state': 1},
  {'pos': [1, 0, 1], 'state': 2},
  {'pos': [2, 0, 0], 'state': 1},
  {'pos': [2, 0, 2], 'state': 1},
  {'pos': [0, 1, 0], 'state': 1},
  {'pos': [0, 1, 1], 'state': 1},
  {'pos': [0, 1, 2], 'state': 1},
  {'pos': [1, 1, 0], 'state': 1},
  {'pos': [1, 1, 1], 'state': 2},
  {'pos': [1, 1, 2], 'state': 1},
  {'pos': [2, 1, 0], 'state': 1},
  {'pos': [2, 1, 1], 'state': 1},
  {'pos': [2, 1, 2], 'state': 1},
  {'pos': [0, 2, 0], 'state': 1},
  {'pos': [0, 2, 2], 'state': 1},
  {'pos': [1, 2, 1], 'state': 3},
  {'pos': [2, 2, 0], 'state': 1},
  {'pos': [2, 2, 2], 'state': 1},
  {'pos': [0, 3, 0], 'state': 1},
 

I can see the value I need is in 'facing'. I'm trying to find out how to best reach this value.

In [24]:
for name in jsonnbt:
    print(name)

size
entities
blocks
palette
DataVersion


An educated guess tells me it's above DataVersion - in 'palette'. 

In [25]:
for name in jsonnbt['palette']:
    print(name)

{'Name': 'minecraft:stone_bricks'}
{'Name': 'minecraft:air'}
{'Properties': {'level': '8'}, 'Name': 'minecraft:lava'}
{'Properties': {'level': '0'}, 'Name': 'minecraft:lava'}
{'Properties': {'waterlogged': 'false', 'facing': 'east', 'type': 'single'}, 'Name': 'minecraft:chest'}


'Facing' inside 'Palette' is stored in a 'Properties' of which the name is 'minecraft:chest'. I'll write a loop that iwll go through 'Palette'. Then it will go through each target-group and check if the 'Name'-value matches 'minecraft:chest'. If so, it will take the 'facing'-value of that group and save it in a new variable named 'direction'. 

In [26]:
for group in jsonnbt['palette']:
    if group['Name'] == 'minecraft:chest':
        direction = group['Properties']['facing']
        print(direction)

east


## The investigation

Now that I know how to get the value out of the NBT-file, it's time to process the Zip-folder. I'll use the zipfile-import. Each NBT-file will be transformed into a JSON-file and subjected to my loop, after which I will save the direction-values in a list. 

In [29]:
import zipfile 

north = []
south = []
west = []
# I'll create 3 empty lists to save the files facing the wrong direction. 

zf = zipfile.ZipFile('Maurons.zip')
with zf as folder:
    for nbtfile in folder.namelist():
        file = nbt.read_from_nbt_file(zf.open(str(nbtfile)))
        jsonfile = file.json_obj(full_json=False)
# The folder is unzipped and a loop runs through each file, turning it into a JSON. 

        for group in jsonfile['palette']:
            if group['Name'] == 'minecraft:chest':
                direction = group['Properties']['facing']
# This is the loop I created earlier, finding the direction of each chest.

                if direction == 'north':
                    north.append(nbtfile[19:22])
                elif direction == 'south':
                    south.append(nbtfile[19:22])
                elif direction == 'west':
                    west.append(nbtfile[19:22])
                elif direction != 'east':
                    print(f'Vreemde waarde in bestand {nbtfile[19:22]}')
# The bottom part looks at the direction-values. For each direction that is not east, it will save the filename. 

## The result

Now I have a list for each wrong direction, and I can let my friend know which files contain a wrong-facing chest.

In [30]:
print(f'Facing north, {len(north)} chests, file names: {north}.\nFacing south, {len(south)} chests, file names: {south}.\nFacing west, {len(west)} chests, file names: {west}.')

Noordelijke kisten, 7 stuks, bestandsnamen: ['059', '112', '150', '571', '611', '694', '708'].
 Zuidelijke kisten, 31 stuks, bestandsnamen: ['033', '062', '093', '117', '172', '216', '261', '263', '274', '287', '292', '321', '340', '390', '445', '493', '524', '527', '600', '629', '646', '647', '754', '798', '826', '872', '878', '926', '986', '994', '996'].
 Westelijke kisten, 6 stuks, bestandsnamen: ['206', '376', '398', '400', '896', '906'].
